# DiploDatos 2019 - Población Penitenciaria en Argentina




> http://diplodatos.famaf.unc.edu.ar/poblacion-penitenciaria-en-argentina-2002-a-2017/




## Práctico Introducción al Aprendizaje Automático

En este práctico utilizaremos algunos algoritmos básicos de aprendizaje automático para intentar encontrar patrones en los datos, y evaluaremos los resultados usando métricas estándares para cada tipo de problema. 

El objetivo del práctico es que pensemos en las distintas formas que puede tomar el modelado de un problema con datos, y en el trabajo de selección de features que debe realizarse para poder abordarlo.

Para ello intentaremos responder la siguiente pregunta:

¿Es posible predecir la duración de la condena que recibirá una persona en base a datos como los que se encuentran presentes en este dataset?

Tener en cuenta los siguientes lineamientos:  

- Utilicen los registros del año 2017 
- Deben decidir a partir de qué valor de una métrica determinada consideran aceptable el resultado. Esto lo tienen que hacer antes de empezar a entrenar un modelo.
- Sólo pueden utilizar las variables que corresponden a información generada antes que la persona ingrese a la institución penal.
- Deberán trabajar con el problema de dos maneras diferentes: como un problema de regresión, y como un problema de clasificación.
- A la hora de hacer selección de variables, tengan en cuenta que los modelos de Machine Learning solo reciben inputs numéricos
- Es importante que almacenen los resultados de alguna manera para poder comparar los distintos modelos. Recomendación: crear una función para almacenar resultados en un dataframe de pandas e imprimirlos en pantalla. 

#### Inicialización y carga de datos

In [291]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

np.random.seed(0)

pd.set_option('display.max_columns', 60)

In [20]:
# Ejemplo, modificar acorde a la ubicación donde tengan su set de datos
data_raw = pd.read_csv('../datasets/sneep_2017_diplodatos.csv', index_col=0)

In [23]:
data_raw.sample(5)

,anio_censo,provincia_descripcion,establecimiento_descripcion,edad,genero_descripcion,nacionalidad_descripcion,estado_civil_descripcion,nivel_instruccion_descripcion,ultima_situacion_laboral_descripcion,capacitacion_laboral_al_ingresar_descripcion,...,es_reincidente_descripcion,tiene_periodo_progresividad_descripcion,reparticion_descripcion,tuvo_salidas_transitorias_descripcion,incorporado_reg_semi_libertad_descripcion,participa_programa_pre_libertad,participa_programa_prision_discontinua_descripcion,participa_programa_semi_detencion_descripcion,tuvo_reduccion_pena_descripcion,mujer_tiene_hijos_intramuro
27706,2017,Buenos Aires,UNIDAD 38 - SIERRA CHICA,37.0,Masculino,Argentina,Casado,Primario incompleto,NaN,Tiene alguna profesión,...,Primario,No se aplica ninguna,Provincial,No le fueron otorgadas,No está incorporado al régimen de semilibertad,1.0,No está incorporado,No tiene semidetención,No tuvo reducción de pena,NaN
77121,2017,Santa Fe,UNIDAD Nº 1 INSTITUTO CORREC. MODELO DR.TABAR...,21.0,Masculino,Argentina,Soltero,Primario incompleto,Desocupado,No tiene ni oficio ni profesión,...,Primario,Período de observación,Provincial,No le fueron otorgadas,No está incorporado al régimen de semilibertad,1.0,No está incorporado,No tiene semidetención,No tuvo reducción de pena,NaN
75262,2017,San Juan,SERVICIO PENITENCIARIO PROVINCIAL CRIO. GRAL. ...,35.0,Masculino,Argentina,Soltero,Primario completo,Desocupado,No tiene ni oficio ni profesión,...,Primario,Período de tratamiento,Provincial,No le fueron otorgadas,No está incorporado al régimen de semilibertad,1.0,No está incorporado,No tiene semidetención,No tuvo reducción de pena,NaN
79643,2017,Santa Fe,UNIDAD Nº11 COMPLEJO PENITENCIARIO DE PIÑERO,22.0,Masculino,Argentina,Soltero,Primario completo,Desocupado,No tiene ni oficio ni profesión,...,NaN,NaN,Provincial,NaN,NaN,0.0,NaN,NaN,NaN,NaN
23422,2017,Buenos Aires,UNIDAD 24 - FLORENCIO VARELA,33.0,Masculino,Argentina,Soltero,Primario completo,Desocupado,No tiene ni oficio ni profesión,...,Reincidente,Período de libertad condicional,Provincial,No le fueron otorgadas,No está incorporado al régimen de semilibertad,1.0,No está incorporado,No tiene semidetención,NaN,NaN


### Regresión

1- Filtrar los registros, conservar sólo aquellos que tengan condena

In [276]:
## Campos que estan relacionados con la condena
for col in data_raw.columns:
    if 'condena' in col:
        print(col)

fecha_condenado
duracion_condena_anios
duracion_condena_meses
duracion_condena_rango
tipo_condena
85283


In [277]:
data_raw.columns

Index(['anio_censo', 'provincia_descripcion', 'establecimiento_descripcion',
       'edad', 'genero_descripcion', 'nacionalidad_descripcion',
       'estado_civil_descripcion', 'nivel_instruccion_descripcion',
       'ultima_situacion_laboral_descripcion',
       'capacitacion_laboral_al_ingresar_descripcion',
       'ultimo_lugar_residencia_descripcion',
       'ultima_provincia_residencia_descripcion', 'jurisdiccion_descripcion',
       'situacion_legal_descripcion', 'fecha_detencion', 'fecha_condenado',
       'establecimiento_procedencia_descripcion', 'delito1_descripcion',
       'delito2_descripcion', 'delito3_descripcion', 'delito4_descripcion',
       'delito5_descripcion', 'horas_trabajo_remunerado_descripcion',
       'participacion_programa_laboral',
       'participacion_programa_educativo_descripcion',
       'participacion_actividades_deportivas',
       'recibio_atencion_medica_ult_anio_descripcion',
       'recibio_visitas_ultimo_anio',
       'participo_alteracion_orde

In [166]:
def preprocess(data):
    return data

def filtrar_condena(data):
    mask = data.tipo_condena.notnull()
    #print(mask)
    filtered_data = data[mask]
    filtered_data.tipo_condena.astype('int')
    return filtered_data

def eliminar_columnas(data):
    drop_cols = [
        'tuvo_reduccion_pena_descripcion',
        'participa_programa_semi_detencion_descripcion',
        'participa_programa_prision_discontinua_descripcion',
        'tiene_periodo_progresividad_descripcion',
        'tuvo_salidas_transitorias_descripcion',
        'incorporado_reg_semi_libertad_descripcion',
        'participa_programa_pre_libertad',
        'situacion_legal_descripcion',
        'participacion_programa_laboral',
        'participacion_programa_educativo_descripcion',
        'participacion_actividades_deportivas',
        'recibio_atencion_medica_ult_anio_descripcion',
        'recibio_visitas_ultimo_anio',
        'participo_alteracion_orden_ult_anio_descripcion',
        'tipo_infraccion_disciplinaria_descripcion',
        'sancion_aplicada_descripcion',
        'calificacion_conducta_descripcion',
        'tentativa_fugas_evasiones_descripcion',
        'tentativa_suicidio',
        'horas_trabajo_remunerado_descripcion',
        'fue_lesionado_descripcion',
        'anio_censo'
    ]
    data = data.drop(drop_cols, axis=1)
    return data

def encode_data(data):
    dummy_cols = [
        'provincia_descripcion'
    ]
    data = pd.get_dummies(data, columns=dummy_cols)
    return data

filtered_data = filtrar_condena(data_raw)
filtered_data = eliminar_columnas(filtered_data)
filtered_data = encode_data(filtered_data)
print(filtered_data.shape)
filtered_data.head()

(43457, 51)


,establecimiento_descripcion,edad,genero_descripcion,nacionalidad_descripcion,estado_civil_descripcion,nivel_instruccion_descripcion,ultima_situacion_laboral_descripcion,capacitacion_laboral_al_ingresar_descripcion,ultimo_lugar_residencia_descripcion,ultima_provincia_residencia_descripcion,jurisdiccion_descripcion,fecha_detencion,fecha_condenado,establecimiento_procedencia_descripcion,delito1_descripcion,delito2_descripcion,delito3_descripcion,delito4_descripcion,delito5_descripcion,duracion_condena_anios,duracion_condena_meses,duracion_condena_rango,tipo_condena,tiene_medidas_seguridad,es_reincidente_descripcion,reparticion_descripcion,mujer_tiene_hijos_intramuro,provincia_descripcion_Buenos Aires,provincia_descripcion_Catamarca,provincia_descripcion_Chaco,provincia_descripcion_Chubut,provincia_descripcion_Ciudad de Buenos Aires,provincia_descripcion_Corrientes,provincia_descripcion_Córdoba,provincia_descripcion_Entre Rios,provincia_descripcion_Formosa,provincia_descripcion_Jujuy,provincia_descripcion_La Pampa,provincia_descripcion_La Rioja,provincia_descripcion_Mendoza,provincia_descripcion_Misiones,provincia_descripcion_Neuquén,provincia_descripcion_Río Negro,provincia_descripcion_Salta,provincia_descripcion_San Juan,provincia_descripcion_San Luis,provincia_descripcion_Santa Cruz,provincia_descripcion_Santa Fe,provincia_descripcion_Santiago del Estero,provincia_descripcion_Tierra del Fuego,provincia_descripcion_Tucumán
0,UNIDAD 13 - JUNIN,28.0,Masculino,Argentina,Soltero,Primario incompleto,Desocupado,No tiene ni oficio ni profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2017-02-23,2017-08-11,Deriv. de una instit. Policial,Otros delitos contra las personas,Amenazas,NaN,NaN,NaN,2.0,2.0,1,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
282,UNIDAD 13 - JUNIN,37.0,Masculino,Argentina,Casado,Ninguno,Desocupado,No tiene ni oficio ni profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2017-02-03,2017-12-03,Deriv. de otro establecimiento,Delitos contra la seguridad pública,NaN,NaN,NaN,NaN,3.0,10.0,2,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
283,UNIDAD 13 - JUNIN,22.0,Masculino,Argentina,Casado,Primario completo,Trabajador de tiempo parcial,Tiene alguna profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2013-03-31,2013-05-14,Deriv. de otro establecimiento,Robo y/o tentativa de robo,NaN,NaN,NaN,NaN,5.0,0.0,2,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
284,UNIDAD 13 - JUNIN,23.0,Masculino,Argentina,Viudo,Primario incompleto,Desocupado,Tiene alguna profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2015-02-02,2016-10-11,Deriv. de otro establecimiento,Homicidios dolosos (tent.),NaN,NaN,NaN,NaN,8.0,7.0,3,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
285,UNIDAD 13 - JUNIN,23.0,Masculino,Paraguaya,Casado,Primario completo,Desocupado,Tiene alguna profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2016-02-24,2017-05-14,Deriv. de otro establecimiento,Infracción ley n° 23.737 (estupefacientes),NaN,NaN,NaN,NaN,4.0,3.0,2,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


2- La variable a predecir que utilizaremos será la condena en meses, generar la feature correspondiente, anexarla al dataset

In [279]:
filtered_data.dropna(subset=['duracion_condena_anios', 'duracion_condena_meses'],inplace=True)
filtered_data['duracion_condena'] = 12*filtered_data['duracion_condena_anios'] + \
                                       filtered_data['duracion_condena_meses']

3- Generar dataframe con las variables de tipo "delito_xx" y la feature generada en el paso anterior. Dividir los datos en conjunto de training y test.

In [280]:
delito_columns = ['delito{:d}_descripcion'.format(i) for i in range(1, 6)]
columns = delito_columns + ['duracion_condena']
data = filtered_data[columns]
df1 = pd.DataFrame(data, columns=columns)
df1

df_delitos = df1.copy()
df_delitos[delito_columns] = df_delitos[delito_columns].astype('category')
cat_columns = df_delitos.select_dtypes(['category']).columns
df_delitos[cat_columns] = df_delitos[cat_columns].apply(lambda x: x.cat.codes)
df_delitos.head()

X = df_delitos[delito_columns]
y = df_delitos['duracion_condena']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

4- Generar un modelo baseline, esto es, un modelo "tonto" que devuelva siempre el promedio de las condenas para el crimen cometido (usaremos solo delito1_id). Calcular 3 métricas de evaluación de regresiones para este modelo. 

In [281]:
X1_train, X1_test = X_train['delito1_descripcion'], X_test['delito1_descripcion']

In [282]:
def predict_condena_naive(X_train, X_test, y_train):
    
    y_pred = np.zeros(X_test.shape[0])
    for i, x in enumerate(X_test):
        mask = X_train == x
        y_pred[i] = y_train[mask].mean()
        
    return y_pred

def compute_metrics(y_true, y_pred):
    mean_sq_err = mean_squared_error(y_true, y_pred)
    mean_abs_err = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    explained_var = explained_variance_score(y_true, y_pred)
    return np.sqrt(mean_sq_err), mean_abs_err, r2

y_pred = predict_condena_naive(X1_train, X1_test, y_train)

compute_metrics(y_test, y_pred)

(60.1461377260645, 42.46140345131342, 0.1361488399061621)

5- Entrenar una regresión lineal y un árbol de decisión usando solo las variables de tipo "delitoxxx". Evaluar métricas para el conjunto de training y para test.

In [283]:
## Linear regressor
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(compute_metrics(y_test, y_pred))

## Tree regressor
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(compute_metrics(y_test, y_pred))

(64.46419835275495, 48.83360378391409, 0.007659755342946273)
(61.35688457376336, 42.4727899730144, 0.10101999580324972)


6- Seleccionar 3 variables adicionales para entrenar los modelos. Generar nuevamente conjuntos de training y test. Entrenar y evaluar los modelos de regresión lineal y árbol de decisión.

In [284]:
### Selecciono variables: edad, nacionalidad y estado civil
variables  = ['edad', 'nacionalidad_descripcion', 'estado_civil_descripcion']
feature_columns = variables + delito_columns
columns = feature_columns + ['duracion_condena']
data = filtered_data[columns]
df2 = pd.DataFrame(data, columns=columns)
df2

df_delitos2 = df2.copy()
df_delitos2[feature_columns] = df_delitos2[feature_columns].astype('category')
cat_columns = df_delitos2.select_dtypes(['category']).columns
df_delitos2[cat_columns] = df_delitos2[cat_columns].apply(lambda x: x.cat.codes)

X = df_delitos2[feature_columns]
y = df_delitos2['duracion_condena']

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Linear regressor
print('Linear regressor')
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

## Tree regressor
print('Desicion tree')
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

Linear regressor
Train (63.55138946372299, 47.29576796986675, 0.04763909241141007)
Test (64.24739252587787, 47.57971292655806, 0.04295676724239483)
Desicion tree
Train (48.24137886013493, 30.54385043840232, 0.45122933869898574)
Test (69.12562173311935, 46.4799065904919, -0.10789510760534893)


### One-hot encoding de delitos

In [285]:
import math
def lista_delitos(df):
    delito_columns = ['delito{:d}_descripcion'.format(i) for i in range(1, 6)]
    df_delito = df[delito_columns]
    
    delitos = pd.unique(df_delito.values.ravel('K')).tolist()
    delitos = list(filter(lambda x: isinstance(x, str), delitos))
    
    n_delitos = len(delitos)
    n_rows = df.shape[0]
    matrix = np.zeros((n_rows, n_delitos), dtype='int')
    delito_dict = {delito: i for i, delito in enumerate(delitos)} 
    
    for i, (index, row) in enumerate(df_delito.iterrows()):
        for delito in row:
            if not isinstance(delito, str):
                continue
            j = delito_dict[delito]
            matrix[i,j] = 1
        
    out_columns = delitos + ['duracion_condena']
    out_df = pd.DataFrame(data=matrix, columns=delitos)
    out_df.index = df.index
    out_df['duracion_condena'] = df['duracion_condena']
    return out_df

new_df = lista_delitos(filtered_data)
new_df.head()

,Otros delitos contra las personas,Delitos contra la seguridad pública,Robo y/o tentativa de robo,Homicidios dolosos (tent.),Infracción ley n° 23.737 (estupefacientes),Amenazas,Violaciones,Homicidios dolosos,Hurto y/o tentativa de hurto,Otros delitos contra la integridad sexual,Privación ilegítima de la libertad,Delitos c/el orden público,Delitos contra el honor,Otros delitos contra la propiedad,Homicidios Culposos,Lesiones Culposas,Lesiones Dolosas,Otros delitos contra la libertad,Delitos contra los poderes publicos,Del. contra la lib. comet. por func. público,Delitos c/ la fe pública,Delitos c/ la administracion pública,Delitos previstos en leyes especiales,Infraccion ley n° 13.944 incumplimiento de deberes,Delitos c/ el estado civil,Contravenciones,Infraccion ley n° 24.769 penal tributaria,Delitos contra la seguridad de la nación,duracion_condena
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.0
282,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.0
283,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60.0
284,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103.0
285,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.0


In [311]:
X = new_df.iloc[:,:-1]
y = new_df['duracion_condena']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Linear regressor
print('Linear regressor')
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

## Tree regressor
print('Desicion tree')
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

Linear regressor
Train (60.700377742057626, 42.48635769770255, 0.14186074020034434)
Test (59.31078857513483, 41.89953738357268, 0.1460262284140118)
Desicion tree
Train (58.739500964156676, 40.574447472309046, 0.19640821381923657)
Test (59.43710262667086, 41.64132297692475, 0.14238494312172845)


### Clasificación

1- Filtrar los registros, conservar sólo aquellos que tengan condena

2- Para poder La variable a predecir debe dividir las condenas en rangos, por ejemplo, las condenas de 1 a 3 años dentro del rango 1, de 3 a 5 en el 2, etc. Analizar el dataset para determinar qué utilizar.

In [313]:
delito_columns = ['delito{:d}_descripcion'.format(i) for i in range(1, 6)]
columns = delito_columns + ['duracion_condena_rango']
data = filtered_data[columns]
df1 = pd.DataFrame(data, columns=columns)
df1

3- Generar dataframe con las variables de tipo "delito_xx" y la feature seleccionada en el paso anterior. Dividir los datos en conjunto de training y test.

In [328]:
df_delitos = df1.copy()
df_delitos[delito_columns] = df_delitos[delito_columns].astype('category')
cat_columns = df_delitos.select_dtypes(['category']).columns
df_delitos[cat_columns] = df_delitos[cat_columns].apply(lambda x: x.cat.codes)
df_delitos.head()

X = df_delitos['delito1_descripcion']
y = df_delitos['duracion_condena_rango']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

4- Generar un modelo baseline, que devuelva el valor más frecuente de rango de condena para el crimen cometido (usaremos solo delito1_id). Calcular 3 métricas de evaluación de regresiones para este modelo.

In [331]:
def classify_condena_naive(X_train, X_test, y_train):
    
    y_pred = np.zeros(X_test.shape[0])
    for i, x in enumerate(X_test):
        mask = X_train == x
        y_pred[i] = np.bincount(y_train[mask]).argmax()
        
    return y_pred

def compute_metrics_classification(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

y_pred = classify_condena_naive(X_train, X_test, y_train)
compute_metrics_classification(y_pred, y_test)

/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.3765659543109801,
 0.6363998402146075,
 0.3765659543109801,
 0.4589630082246017)

5- Entrenar una regresión logística y un árbol de decisión usando solo las variables de tipo "delitoxxx". Evaluar métricas para el conjunto de training y para test.

In [338]:
X = df_delitos[delito_columns]
y = df_delitos['duracion_condena_rango']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)
print('Train:', compute_metrics_classification(y_pred, y_train))

y_pred = model.predict(X_test)
print('Test: ', compute_metrics_classification(y_pred, y_test))

Train: (0.41755711127487105, 0.5776760352425012, 0.41755711127487105, 0.46483023656498945)
Test:  (0.3858695652173913, 0.5443298320394334, 0.3858695652173913, 0.4376811050049932)


6- Seleccionar 3 variables adicionales para entrenar los modelos. Generar nuevamente conjuntos de training y test. Entrenar y evaluar los modelos de regresión lineal y árbol de decisión.

### Ajuste de Hiperparámetros

1- Elegir un modelo (puede ser de regresión o de clasificación) y realizar ajuste de hiperparámetros. Usar grid search y cross-validation. Reportar resultados.

In [312]:
for idx, criterion in enumerate((['mse']), start=1):
    exploring_params = {
        #'learning_rate': ['constant'],
        'splitter': ['best', 'random'],
        'max_depth': [1, 2, 3, 4, None],  
        'min_samples_leaf': [1, 10, 20, 50, 100] 
    }
    ad = DecisionTreeRegressor(criterion=criterion, random_state=0)
    model = GridSearchCV(ad, exploring_params, cv=5, scoring='neg_mean_squared_error')
    model.fit(X_train, y_train)
    
    print("# Exploración de hiperparámetros para criterion \"%s\"" % criterion, end="\n\n")
    
    print("Mejor conjunto de parámetros:")
    print(model.best_params_, end="\n\n")

    print("Puntajes de la grilla:", end="\n\n")
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, model.cv_results_['params']):
        print("Exactitud: %0.4f (+/-%0.04f) para los parámetros %r" % (np.sqrt(-mean), np.sqrt(std), params))
    print()

    print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
    y_true, y_pred = y_test, model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print('RMSE = {:.4f}'.format(rmse), end='\n\n')
    print('Mejores parametros:')
    print(model.best_params_)
    print("================================================", end="\n\n")

# Exploración de hiperparámetros para criterion "mse"

Mejor conjunto de parámetros:
{'max_depth': None, 'min_samples_leaf': 20, 'splitter': 'best'}

Puntajes de la grilla:

Exactitud: 63.8023 (+/-9.0452) para los parámetros {'max_depth': 1, 'min_samples_leaf': 1, 'splitter': 'best'}
Exactitud: 63.8023 (+/-9.0452) para los parámetros {'max_depth': 1, 'min_samples_leaf': 1, 'splitter': 'random'}
Exactitud: 63.8023 (+/-9.0452) para los parámetros {'max_depth': 1, 'min_samples_leaf': 10, 'splitter': 'best'}
Exactitud: 63.8023 (+/-9.0452) para los parámetros {'max_depth': 1, 'min_samples_leaf': 10, 'splitter': 'random'}
Exactitud: 63.8023 (+/-9.0452) para los parámetros {'max_depth': 1, 'min_samples_leaf': 20, 'splitter': 'best'}
Exactitud: 63.8023 (+/-9.0452) para los parámetros {'max_depth': 1, 'min_samples_leaf': 20, 'splitter': 'random'}
Exactitud: 63.8023 (+/-9.0452) para los parámetros {'max_depth': 1, 'min_samples_leaf': 50, 'splitter': 'best'}
Exactitud: 63.8023 (+/-9.0452) para lo

2- Responder: ¿Mejoró sustancialmente el rendimiento de los modelos? ¿Por qué?

### OPCIONAL: Encoding de variables categóricas

Si tienen tiempo, lean el siguiente link: https://www.datacamp.com/community/tutorials/categorical-data.
Y luego ejecuten los siguientes puntos:


1- Usar la función get_dummies() de Pandas para obtener one-hot encoding de las variables categóricas

2- Seleccionar un modelo de regresión o clasificación y evaluar los resultados con las nuevas variables